In [1]:
import librosa
from librosa import display
import matplotlib.pyplot as plt
import scipy
import os
import tensorflow as tf
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Dense
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.activations import sigmoid, relu, softmax
from keras.models import Sequential
from keras.optimizers import rmsprop, Adam,RMSprop
from keras.preprocessing.image import ImageDataGenerator
import csv
import pandas as pd
import numpy as np
from numpy import array

Using TensorFlow backend.


# Preprocessing of Data


In [2]:
path_directory_training= r'C:\Users\Gebruiker\Desktop\training\images'
path_labels_training= r'C:\Users\Gebruiker\Desktop\training\labels.csv'

path_labels_valid =  r'C:\Users\Gebruiker\Desktop\valid\labels.csv'
path_directory_valid =  r'C:\Users\Gebruiker\Desktop\valid\images'

path_directory_test = r'C:\Users\Gebruiker\Desktop\test\images'
path_labels_test =  r'C:\Users\Gebruiker\Desktop\test\labels.csv'

In [4]:
df_training = pd.read_csv(path_labels_training)

df_valid = pd.read_csv(path_labels_valid)

df_test = pd.read_csv(path_labels_test)

df_LOL = pd.read_csv(r'C:\Users\Gebruiker\Desktop\Pitch-labels_GUITARS.csv')

datagenVal = ImageDataGenerator()
datagenTrain = ImageDataGenerator(rescale = 1./255,width_shift_range=0.2)

train_generator = datagenTrain.flow_from_dataframe(
df_training,
directory=path_directory_training,
x_col='filename',
y_col=class_list,
batch_size=32,
target_size=(100,100),
class_mode='raw',
shuffle= True,
color_mode ='grayscale')

validation_generator = datagenVal.flow_from_dataframe(
df_valid,
directory=path_directory_valid,
x_col='filename',
y_col=class_list,
batch_size=32,
target_size=(100,100),
class_mode='raw',
shuffle= False,
color_mode='grayscale')


datagenTest = ImageDataGenerator()

test_generator = datagenTest.flow_from_dataframe(
df_test, 
directory = path_directory_test,    
x_col = 'filename',
y_col = class_list,
class_mode = 'raw',
color_mode='grayscale',
target_size=(100,100),
shuffle= False,
batch_size = 32)

evaluate_generator = datagenTest.flow_from_dataframe(
dataframe = df_LOL,
directory= r'C:\Users\Gebruiker\Desktop\test.png',
batch_size= 1,
target_size=(100,100),
class_mode= None,
shuffle= False,
color_mode='grayscale')

training_steps = train_generator.n // train_generator.batch_size
validation_steps = validation_generator.n // validation_generator.batch_size
test_steps = test_generator.n // test_generator.batch_size
evaluate_steps = evaluate_generator.n // evaluate_generator.batch_size


FileNotFoundError: [Errno 2] File C:\Users\Gebruiker\Desktop\training\labels.csv does not exist: 'C:\\Users\\Gebruiker\\Desktop\\training\\labels.csv'

# Building of model

In [1]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu',input_shape=(100,100,1)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(88, activation='softmax'))

model.compile(loss=categorical_crossentropy,metrics=['accuracy'],optimizer= RMSprop(lr=1e-3))


NameError: name 'Sequential' is not defined

# Training of model

In [47]:
train_generator.reset()
validation_generator.reset()
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=training_steps,
                    epochs=5,validation_data=validation_generator,
                    validation_steps=validation_steps,)

model.save( r'C:\Users\Gebruiker\Desktop\modelAugmentedGereator1.h5')

Epoch 1/5
623/623 [==============================] - 587s 942ms/step - loss: 2.2196 - accuracy: 0.4947 - val_loss: 135.4628 - val_accuracy: 0.7279
Epoch 2/5
623/623 [==============================] - 287s 461ms/step - loss: 1.1504 - accuracy: 0.7162 - val_loss: 125.3368 - val_accuracy: 0.8034
Epoch 3/5
623/623 [==============================] - 309s 496ms/step - loss: 0.8846 - accuracy: 0.7706 - val_loss: 120.2811 - val_accuracy: 0.8284
Epoch 4/5
623/623 [==============================] - 256s 412ms/step - loss: 0.7459 - accuracy: 0.8015 - val_loss: 134.8374 - val_accuracy: 0.8221
Epoch 5/5
623/623 [==============================] - 258s 415ms/step - loss: 0.6421 - accuracy: 0.8229 - val_loss: 150.7789 - val_accuracy: 0.8413


# Evaluation of model

In [73]:
## See if the output with their corresponding images , pitch      ONLY FOR 32 FIRST IMAGES
test_generator.reset() ## reset the iterator to begin of the loop again
listPredictedNotes = []

noteDictionary = {}

for index,note in enumerate(class_list):  ## transforming onehot-encoded to their classes (notes)
    noteDictionary[index] = note
    
model = tf.keras.models.load_model(r'C:\Users\Gebruiker\Desktop\modelAugmentedGereator1.h5') #load saved model
predictions = model.predict_generator(test_generator,steps = test_steps,verbose = 1)

for prediction in predictions.tolist():
    try:
        indexOutput = prediction.index(1)
    except:
        listPredictedNotes.append('No output')    
    else:
        listPredictedNotes.append(noteDictionary.get(indexOutput))
       
for indexPrediction,output in enumerate(listPredictedNotes):
    print(f'Prediction {indexPrediction + 1} : {output}')

20/20 [==============================] - 2s 124ms/step
Prediction 1 : A#4
Prediction 2 : D6
Prediction 3 : G#4
Prediction 4 : F#6
Prediction 5 : D2
Prediction 6 : F1
Prediction 7 : F#7
Prediction 8 : A2
Prediction 9 : A7
Prediction 10 : C7
Prediction 11 : G7
Prediction 12 : F3
Prediction 13 : No output
Prediction 14 : E7
Prediction 15 : E4
Prediction 16 : A#0
Prediction 17 : F2
Prediction 18 : C#4
Prediction 19 : E3
Prediction 20 : B3
Prediction 21 : F5
Prediction 22 : D5
Prediction 23 : E4
Prediction 24 : E2
Prediction 25 : G4
Prediction 26 : F2
Prediction 27 : F3
Prediction 28 : A5
Prediction 29 : F#2
Prediction 30 : G#4
Prediction 31 : D3
Prediction 32 : A#2
Prediction 33 : G7
Prediction 34 : C7
Prediction 35 : B4
Prediction 36 : E5
Prediction 37 : F4
Prediction 38 : No output
Prediction 39 : C2
Prediction 40 : E4
Prediction 41 : A0
Prediction 42 : G#2
Prediction 43 : A3
Prediction 44 : G#7
Prediction 45 : F#1
Prediction 46 : G#3
Prediction 47 : F5
Prediction 48 : D#2
Prediction 49 

Prediction 507 : D4
Prediction 508 : C#2
Prediction 509 : A#0
Prediction 510 : C5
Prediction 511 : B5
Prediction 512 : G4
Prediction 513 : B4
Prediction 514 : B1
Prediction 515 : F6
Prediction 516 : B1
Prediction 517 : F#6
Prediction 518 : G3
Prediction 519 : A#2
Prediction 520 : C5
Prediction 521 : C#7
Prediction 522 : G1
Prediction 523 : C#2
Prediction 524 : E7
Prediction 525 : C#4
Prediction 526 : C1
Prediction 527 : G#3
Prediction 528 : G#6
Prediction 529 : D#3
Prediction 530 : B0
Prediction 531 : D#7
Prediction 532 : B2
Prediction 533 : D#3
Prediction 534 : C7
Prediction 535 : F7
Prediction 536 : A#7
Prediction 537 : B5
Prediction 538 : F5
Prediction 539 : F#2
Prediction 540 : F5
Prediction 541 : C#3
Prediction 542 : B4
Prediction 543 : G2
Prediction 544 : A#4
Prediction 545 : E7
Prediction 546 : C1
Prediction 547 : C8
Prediction 548 : C4
Prediction 549 : F1
Prediction 550 : E2
Prediction 551 : G2
Prediction 552 : No output
Prediction 553 : D#3
Prediction 554 : D7
Prediction 555 :

In [92]:
model1 = tf.keras.models.load_model(r'C:\Users\Gebruiker\Desktop\modelAugmentedGenerator1.h5')
model2 = tf.keras.models.load_model(r'C:\Users\Gebruiker\Desktop\modelv5.h5')

test_generator.reset()
loss, accuracy = model2.evaluate(x = test_generator,steps= test_steps)


20/20 [==============================] - 3s 138ms/step - loss: 102.1143 - accuracy: 0.8328


In [1]:
def generateSpectogram(sourcePath):     #generate Mel-spectogram from wav 
    plt.box(0)
    samples, sampling_rate = librosa.load(sourcePath)
    spectogram = librosa.stft(samples)
    mel = librosa.feature.melspectrogram(y=samples,sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel)) 
    librosa.display.specshow(mel_dB,cmap='gray')
    plt.savefig(r'C:\Users\Gebruiker\Desktop\test.png')
    plt.close()
    
    return samples

In [53]:
evaluate_generator.reset()
result = model.predict_generator(evaluate_generator,evaluate_steps)
print(result)

[[2.7300245e-30 1.0000000e+00 3.2943802e-24 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000